### 로컬에서 사용할 수 있는 마이크 출력

In [5]:
import sounddevice as sd

def list_microphones():
    """
    List available microphones and their corresponding IDs.
    """
    devices = sd.query_devices()
    mic_list = []
    
    for idx, device in enumerate(devices):
        if device['max_input_channels'] > 0:
            print(f"ID: {idx}, Name: {device['name']}, Channels: {device['max_input_channels']}")
            mic_list.append((idx, device['name']))
    
    return mic_list

# Get the list of microphones
microphones = list_microphones()


ID: 0, Name: Microsoft Sound Mapper - Input, Channels: 2
ID: 1, Name: 마이크(Realtek(R) Audio), Channels: 2
ID: 4, Name: 주 사운드 캡처 드라이버, Channels: 2
ID: 5, Name: 마이크(Realtek(R) Audio), Channels: 2
ID: 9, Name: 마이크(Realtek(R) Audio), Channels: 2
ID: 12, Name: PC 스피커 (Realtek HD Audio 2nd output with SST), Channels: 2
ID: 13, Name: 스테레오 믹스 (Realtek HD Audio Stereo input), Channels: 2
ID: 16, Name: PC 스피커 (Realtek HD Audio output with SST), Channels: 2
ID: 17, Name: 마이크 (Realtek HD Audio Mic input), Channels: 2


In [21]:
import os
import shutil  # 추가: 디렉터리를 비우는 데 사용
import numpy as np
import sounddevice as sd
from scipy.io import wavfile as wav
from pydub import AudioSegment
import librosa
from tensorflow.keras.models import load_model

def record_audio(duration=20, fs=16000, filename='record_demo.wav'):
    print("Recording audio for", duration, "seconds...")
    myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    wav.write(filename, fs, myrecording)
    print("Length of recording:", len(myrecording) / fs, "seconds")  # 추가: 레코딩 길이 확인
    print("Recording is finished")
    return filename

def clear_directory(directory_path):
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)  # 디렉터리 삭제
    os.makedirs(directory_path)  # 디렉터리 생성

def audio_chunking(filename):
    output_dir = "./5sec_cut_result_demo/"
    clear_directory(output_dir)  # 추가: 디렉터리 비우기
    audio = AudioSegment.from_wav(filename)
    length_audio = len(audio)
    start = 0
    end = 0
    counter = 1

    while start < length_audio:
        end += 5000
        if end > length_audio:
            end = length_audio
        chunk = audio[start:end]
        if len(chunk) == 5000:
            chunk.export(output_dir + f"{os.path.splitext(filename)[0]}_part_{counter:04d}.wav", format="wav")
            counter += 1
        start += 5000

    return output_dir



def handle_dB_NaN(dB, threshold=-80):
    NAN = False
    B_num = 0
    B_nan = 0
    
    for i in range(len(dB)):
        if NAN == False and (dB[i] != threshold):
            B_num = dB[i]
        elif NAN == True and (dB[i] != threshold):
            avg = (B_num + dB[i]) / 2
            for j in range(B_nan, i):
                dB[j] = avg
            NAN = False
            B_num = dB[i]
        elif (dB[i] == threshold) and NAN == False:
            if i == len(dB) - 1:
                dB[i] = B_num / 2
            NAN = True
            B_nan = i
        elif (dB[i] == threshold) and i == len(dB) - 1:
            avg = B_num / 2
            for j in range(B_nan, i + 1):
                dB[j] = avg
                
    return dB


def dB_npy(output_dir):
    errorLogPath = "dB_error_log.txt"
    output_folder = './dB_result_demo/'
    clear_directory(output_folder)  # 추가: 디렉터리 비우기

    file_list = os.listdir(output_dir)
    file_list_wav = [file for file in file_list if file.endswith('.wav')]

    error_log = []
    
    for filename in file_list_wav:
        audio_data, sr = librosa.load(os.path.join(output_dir, filename), sr=16000)
        rms_values = librosa.feature.rms(y=audio_data)[0]
        dB = librosa.power_to_db(rms_values, ref=np.max)
        
        processed_dB = handle_dB_NaN(dB)
        
        processed_dB = processed_dB.astype(np.float32)
        folder_num = len(os.listdir(output_folder))
        file_path = os.path.join(output_folder, 'dB_demo_' + str(folder_num))
        np.save(file_path, processed_dB)
    
        if -80 in processed_dB:
            error_log.append(filename)

    with open(errorLogPath, "w") as f:
        for filename in error_log:
            f.write(f"{filename}\n")

            
def load_saved_model(model_path):
    model_best = load_model(model_path)
    model_best.summary()
    return model_best

def preprocess_files(file_list_pitch, path):
    reshaped_files = []
    
    for file in file_list_pitch:
        file_path = os.path.join(path, file)
        if os.path.exists(file_path):
            data = np.load(file_path)
            reshaped_data = data.reshape(1, -1, 1) # Data reshaping, flexible to the time dimension
            reshaped_files.append(reshaped_data)
        else:
            print(f"File {file_path} not found.")
    
    if reshaped_files:
        return np.vstack(reshaped_files).astype('float32')
    else:
        print("No data to stack.")
        return np.array([])

def predict_with_model(model_best, file_list_pitch, path):
    data = preprocess_files(file_list_pitch, path)
    
    if data.size > 0:
        probabilities = model_best.predict(data)
        labels = [1 if prob[0] >= 0.5 else 0 for prob in probabilities]
        print("Probabilities and Labels for Each File:")
        
        # 각 파일마다 예측 확률과 레이블을 출력합니다.
        for file_name, prob, label in zip(file_list_pitch, probabilities, labels):
            print(f"File {file_name}: Probability for class 0: {1 - prob[0]:.4f}, "
                  f"Probability for class 1: {prob[0]:.4f}, Label: {label}")
            
        return labels, probabilities
    else:
        print("No valid data to predict.")
        return None, None

def evaluate_voice_phishing_risk(probabilities, labels):
    num_voice_phishing_files = 0
    sum_class_0_prob_voice_phishing = 0.0

    for idx, label in enumerate(labels):
        if label == 0 and 1 - probabilities[idx][0] >= 0.6:  
            num_voice_phishing_files += 1
            sum_class_0_prob_voice_phishing += 1 - probabilities[idx][0]

    total_num_files = len(labels)
    if num_voice_phishing_files > 0:
        probability_P = (num_voice_phishing_files / total_num_files) * (sum_class_0_prob_voice_phishing / num_voice_phishing_files)
    else:
        probability_P = 0.0

    percentage_P = probability_P * 100  # Convert to percentage

    if 0 <= percentage_P < 60:
        risk_level = "보이스피싱 안전"
    elif 60 <= percentage_P < 80:
        risk_level = "보이스피싱 위험"
    elif 80 <= percentage_P <= 100:
        risk_level = "보이스피싱 매우 위험"
    else:
        risk_level = "정상"

    print("Number of voice phishing files:", num_voice_phishing_files)
    print("Total number of files:", total_num_files)
    print("Probability P:", probability_P)
    print("Risk Level:", risk_level)
    
def main():
    filename = record_audio()
    output_dir = audio_chunking(filename)
    dB_npy(output_dir)
    
    model_best = load_saved_model('D:\\user\Downloads\\시연_decibel_model.h5')

    path = 'C:\\Users\\user\\Jinwoo\\dB_result_demo\\'
    file_list = os.listdir(path)
    file_list_pitch = [file for file in file_list if file.endswith('.npy')]
    
    print("\n=== Predictions for decibel data ===\n")
    labels, probabilities = predict_with_model(model_best, file_list_pitch, path)
    
    if labels is not None and probabilities is not None:
        evaluate_voice_phishing_risk(probabilities, labels)

if __name__ == '__main__':
    main()

Recording audio for 20 seconds...
Length of recording: 20.0 seconds
Recording is finished
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 157, 1)]          0         
                                                                 
 normalization (Normalizatio  (None, 157, 1)           315       
 n)                                                              
                                                                 
 Conv1D_1 (Conv1D)           (None, 157, 32)           128       
                                                                 
 Conv1D_2 (Conv1D)           (None, 157, 64)           6208      
                                                                 
 Conv1D_3 (Conv1D)           (None, 157, 128)          24704     
                                                                 
 Conv1D_4 (Conv1D)           (None, 1